In [ ]:
from typing import TypedDict, Sequence, Union, Annotated
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage, ToolMessage
from langchain_openai import AzureChatOpenAI
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
import os


openai_endpoint = 'https://learningoai-si.openai.azure.com/'#os.getenv("openai_endpoint", "https://your-resource-name.openai.azure.com/")
openai_apikey = os.getenv("openai_apikey", "{your-api-key}")
#openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview
print(f"Using OpenAI Endpoint: {openai_endpoint}")

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    
    

c:\Users\manoj.g\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using OpenAI Endpoint: https://learningoai-si.openai.azure.com/


In [13]:
@tool
def add(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers together."""
    return a * b

tools = [add, multiply]


llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    azure_endpoint=openai_endpoint,
    api_version="2025-01-01-preview",
    api_key=openai_apikey).bind_tools(tools)

In [4]:
def model_calls(state: AgentState) -> AgentState:
    """This node will solve the request you input"""
    system_prompt = SystemMessage(content="You are a helpful AI assistant.")
    response = llm.invoke([system_prompt] + state["messages"])
    return {"messages": [response]}

In [5]:
def should_continue(state: AgentState) -> str:
    """Decide whether to continue the conversation based on the last AI message."""
    last_message = state["messages"][-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"

In [15]:
graph = StateGraph(AgentState)
graph.add_node("our_agent", model_calls)

tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

graph.set_entry_point("our_agent")
graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    })
graph.add_edge("tools", "our_agent")

app = graph.compile()

In [11]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [16]:
inputs = {"messages":[{"role":"user","content":"Add 40+12 and then multiply the result by 3. Also tell me a joke please"}]}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Add 40+12 and then multiply the result by 3. Also tell me a joke please
================================== Ai Message ==================================
Tool Calls:
  add (call_BdjpqeWBweRasXzqI5gMzBKO)
 Call ID: call_BdjpqeWBweRasXzqI5gMzBKO
  Args:
    a: 40
    b: 12
================================= Tool Message =================================
Name: add

52
================================== Ai Message ==================================
Tool Calls:
  multiply (call_wcMTmBbthIoOHwaFn1e04RmY)
 Call ID: call_wcMTmBbthIoOHwaFn1e04RmY
  Args:
    a: 52
    b: 3
================================= Tool Message =================================
Name: multiply

156
================================== Ai Message ==================================

The result of adding 40 + 12 and then multiplying the result by 3 is 156.

And here's a joke for you:
Why did the math book look sad?  
Because it had too many proble